## Importing libraries and installing xlrd for reading xlsx files

In [1]:
import numpy as np
import pandas as pd
import os

## Reading xlsx files and joining them together
#### Please download all excel files and put in same directory as your current notebook

In [2]:
path = os.getcwd()
filename = '\\body_composition_measurements_subject_'
subjects = ['A', 'B', 'C', 'D', 'E']

df = pd.DataFrame()
for subject in subjects:
    dftemp = pd.read_excel('{}{}{}.xlsx'.format(path, filename, subject))
    df = df.append(dftemp, ignore_index=True)

df.shape

(186, 11)

### Add a column for recognizing veganism vs. normal diet and rename a column

In [3]:
df['vegan'] = np.where(df['Date (yy-mm-dd)'] > '2020-10-22', True, False)
df = df.rename(columns={"Date (yy-mm-dd)": "Date"})

### Save merged file as a excel

In [4]:
# save merged files as a one file
df.to_excel('df_BC.xlsx')

### Ploting interactive panel

In [23]:
import panel as pn
import hvplot.pandas
from panel.interact import interact

xs = [0.1* i for i in range(30)]

pn.extension()

subs = ['A', 'B', 'C', 'D', 'E']
graf = ['Line', 'Boxplot', 'Violin']
var = pn.widgets.ToggleGroup(options=subs, behavior='radio')
var2 = pn.widgets.ToggleGroup(options=graf, behavior='radio')

@pn.depends(var, var2)
def tabs(sub, graph):
    dft = df[df['Subject'] == sub]
    if graph == 'Line':
        return pn.Tabs(
            ('Weight' , dft.hvplot.scatter(x='Date', y='Bodyweight (kg)', by='vegan', s=100, height=400, width=800)*dft.hvplot.line(x='Date', y='Bodyweight (kg)')),
            ('BMI',     dft.hvplot.scatter(x= 'Date', y='BMI (score)', by='vegan', s=100, height=400, width=800)*dft.hvplot.line(x='Date', y='BMI (score)')),
            ('Bodyfat', dft.hvplot.scatter(x='Date', y='Bodyfat (%)', by='vegan', s=100, height=400, width=800)*dft.hvplot.line(x='Date', y='Bodyfat (%)')),
            ('Muscle',  dft.hvplot.scatter(x='Date', y='Skeletal muscle (%)', by='vegan', s=100, height=400, width=800)*dft.hvplot.line(x='Date', y='Skeletal muscle (%)')))
    elif graph == 'Boxplot':
        return pn.Tabs(
            ('Weight' , dft.hvplot.box(y='Bodyweight (kg)', by='vegan', color='vegan',cmap=['blue', 'orange'], legend=False, height=400, width=800)),
            ('BMI',     dft.hvplot.box(y='BMI (score)', by='vegan', color='vegan',cmap=['blue', 'orange'], legend=False, height=400, width=800)),
            ('Bodyfat', dft.hvplot.box(y='Bodyfat (%)', by='vegan', color='vegan',cmap=['blue', 'orange'], legend=False, height=400, width=800)),
            ('Muscle',  dft.hvplot.box(y='Skeletal muscle (%)', by='vegan', color='vegan',cmap=['blue', 'orange'], legend=False, height=400, width=800)))
    elif graph == 'Violin':
        return pn.Tabs(
            ('Weight' , dft.hvplot.violin(y='Bodyweight (kg)', by='vegan' ,padding=0.5, color='vegan',cmap=['blue', 'orange'], height=400, width=800)),
            ('BMI',     dft.hvplot.violin(y='BMI (score)', by='vegan' ,padding=0.5, color='vegan',cmap=['blue', 'orange'], height=400, width=800)),
            ('Bodyfat', dft.hvplot.violin(y='Bodyfat (%)', by='vegan' ,padding=0.5, color='vegan',cmap=['blue', 'orange'], height=400, width=800)),
            ('Muscle',  dft.hvplot.violin(y='Skeletal muscle (%)', by='vegan' ,padding=0.5, color='vegan',cmap=['blue', 'orange'], height=400, width=800)))

interact(tabs, sub=var, graph=var2)

Column
    [0] Column
        [0] RadioButtonGroup(options=['A', 'B', 'C', 'D', 'E'], value='A')
        [1] RadioButtonGroup(options=['Line', 'Boxplot', ...], value='Line')
    [1] Row
        [0] Tabs
            [0] HoloViews(Overlay, name='Weight')
            [1] HoloViews(Overlay, name='BMI')
            [2] HoloViews(Overlay, name='Bodyfat')
            [3] HoloViews(Overlay, name='Muscle')